In [1]:
!pip install paddleocr
!pip install paddlepaddle
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install PyMuPDF
!pip install spacy openpyxl
!pip install gliner-spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.7/544.7 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 14.8 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=908942009d6822af4eacdf65580691411582cffe2348403e77752094d3a3f485
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire
  Attempting uninstall: albucore
    F

In [2]:
from paddleocr import PaddleOCR
import spacy
from gliner_spacy.pipeline import GlinerSpacy
import openpyxl
from openpyxl.styles import Font

def extract_text_from_image(image_path):
    """
    Extracts text from an image using PaddleOCR.

    Parameters:
        image_path (str): Path to the image file.

    Returns:
        str: Extracted text concatenated into a single string.
    """
    # Initialize PaddleOCR with angle classification and English language support
    ocr = PaddleOCR(use_angle_cls=True, lang='en')

    # Perform OCR on the image to extract text
    results = ocr.ocr(image_path, cls=True)

    # Extract text from OCR results and store it in a list
    extracted_text = []
    for line in results[0]:
        extracted_text.append(line[1][0])  # Append the recognized text to the list

    # Return the extracted text as a single concatenated string
    return "\n".join(extracted_text)

def extract_personal_details(text):
    """
    Extracts personal details using GlinerSpacy.

    Parameters:
        text (str): Input text to process.

    Returns:
        dict: Extracted personal details categorized by label.
    """
    # Load the spaCy model and add the GlinerSpacy pipeline for personal details extraction
    nlp = spacy.load("en_core_web_sm")
    nlp.add_pipe("gliner_spacy", config={"labels": [
        "name", "phone no.", "old", "email", "school education",
        "role", "college", "year graduated", "birthdate"
    ]})

    # Process the input text using the spaCy model
    doc = nlp(text)

    # Create a dictionary to store extracted details
    personal_details = {}
    for ent in doc.ents:
        # Only take the first occurrence of each label
        if ent.label_ not in personal_details:
            if ent.label_ == "old":  # Special handling for "old" (age)
                cleaned_age = "".join(ent.text.split())  # Remove spaces from age
                if len(cleaned_age) > 2:  # Limit age length to two digits
                    cleaned_age = cleaned_age[:2]
                personal_details[ent.label_] = cleaned_age
            else:
                personal_details[ent.label_] = ent.text.strip()  # Store the entity text

    return personal_details

def save_to_excel_rowwise(details, output_file):
    """
    Saves extracted details to an Excel file in a row-wise format.

    Parameters:
        details (dict): Extracted details categorized by label.
        output_file (str): Path to the Excel file to save data.
    """
    # Create a new Excel workbook and select the active sheet
    workbook = openpyxl.Workbook()
    sheet = workbook.active

    # Write the headers (keys of the details dictionary) to the first row
    headers = list(details.keys())
    sheet.append(headers)

    # Make the first row text bold
    bold_font = Font(bold=True)
    for cell in sheet[1]:
        cell.font = bold_font

    # Write the values (values of the details dictionary) to the second row
    sheet.append(list(details.values()))

    # Save the workbook to the specified output file
    workbook.save(output_file)
    print(f"Details saved to Excel file: {output_file}")

if __name__ == "__main__":
    # Path to the image file
    image_path = "/content/test_form.png"  # Replace with the path to your uploaded image

    # Output Excel file name
    output_excel_file = "extracted_details_rowwise.xlsx"

    # Step 1: Extract text from image using PaddleOCR
    extracted_text = extract_text_from_image(image_path)
    print("Extracted Text from Image:")
    print(extracted_text)

    # Step 2: Extract personal details using GlinerSpacy
    print("\nExtracted Personal Details:")
    personal_details = extract_personal_details(extracted_text)

    # Print the extracted personal details
    for label, entity in personal_details.items():
        print(f"{label}: {entity}")

    # Step 3: Save extracted details to an Excel file with bold first row
    save_to_excel_rowwise(personal_details, output_excel_file)


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:17<00:00, 226.27it/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10000/10000 [00:20<00:00, 477.97it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


...

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


name: Marion R
birthdate: 02/19/2055
email: marionlyons@email.com
phone no.: 222 555 7777
role: Junior Writer
school education: High School
college: N.Armatrong College
year graduated: 2075
Details saved to Excel file: extracted_details_rowwise.xlsx
